## Voy a probar usando superficie cubierta y superficie total

In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.externals import joblib
import time
from sklearn.model_selection import train_test_split

import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

In [2]:
propiedades = pd.read_csv('../../set_datos_superficies.csv')

sup_min = 10
sup_max = 300
lat_min = -35
lat_max = -34
lon_min = -59
lon_max = -58
precio_min = 30000
precio_max = 3000000

propiedades = propiedades.loc[(propiedades.price_aprox_usd.notnull()) \
                              & (propiedades.lat.notnull()) & (propiedades.lon.notnull()) \
                        & (propiedades.price_aprox_usd <= precio_max) & (propiedades.price_aprox_usd >= precio_min) &\
                         (propiedades.lat <= lat_max) & (propiedades.lat >= lat_min) &\
                       (propiedades.lon <= lon_max) & (propiedades.lon >= lon_min),:]

In [3]:
set_pruebas = pd.read_csv('../properati_dataset_modificado2.csv')
set_pruebas.loc[:,'price_usd'] = 0

def normalizar_sup(s):
    if not s:
        return None
    if s < sup_min:
        return sup_min
    if s > sup_max:
        return sup_max
    return s

set_pruebas.loc[:,'surface_covered_in_m2'] = set_pruebas.loc[:,'surface_covered_in_m2'].apply(lambda x: normalizar_sup(x))
set_pruebas.loc[:,'surface_total_in_m2'] = set_pruebas.loc[:,'surface_total_in_m2'].apply(lambda x: normalizar_sup(x))

# Prueba

In [3]:
columnas = ['surface_covered_in_m2','surface_total_in_m2','lat', 'lon','property_type_encoded','gimnasio']
cond_prop = (propiedades.surface_covered_in_m2.notnull()) & (propiedades.surface_total_in_m2.notnull()) &\
                    (propiedades.surface_covered_in_m2 <= sup_max) & (propiedades.surface_covered_in_m2 >= sup_min) &\
            (propiedades.surface_total_in_m2 <= sup_max) & (propiedades.surface_total_in_m2 >= sup_min) 
        
X_train, X_test, y_train, y_test = train_test_split(propiedades.loc[cond_prop,columnas], \
                                        propiedades.loc[cond_prop,'price_aprox_usd'], test_size=0.2)

In [4]:
tree = DecisionTreeRegressor(criterion = 'mse', max_depth = 100, max_features = 0.8)
    
tree.fit(X_train,y_train)
precision = tree.score(X_test,y_test) * 100
error = mean_squared_error(tree.predict(X_test), y_test)
print("Precision = {:.2f},  error = {:.0f}, dig = {}".format(precision,error,len(str(int(round(error,0))))))

Precision = 97.11,  error = 1264832770, dig = 10


# Calculo para aquellos que tienen tanto superficie total como cubierta

In [4]:
condiciones = (set_pruebas.surface_covered_in_m2.notnull()) & (set_pruebas.surface_total_in_m2.notnull())
cond_prop = (propiedades.surface_covered_in_m2.notnull()) & (propiedades.surface_total_in_m2.notnull()) &\
                    (propiedades.surface_covered_in_m2 <= sup_max) & (propiedades.surface_covered_in_m2 >= sup_min) &\
            (propiedades.surface_total_in_m2 <= sup_max) & (propiedades.surface_total_in_m2 >= sup_min) 


In [5]:
cant =15
columnas = ['surface_covered_in_m2','surface_total_in_m2','lat', 'lon','property_type_encoded','gimnasio']

for i in range(cant):
    datos = propiedades.loc[cond_prop,:].sample(frac = 0.8)
    set_entrenamiento_datos = datos.loc[:,columnas]
    set_entrenamiento_resultado = datos.loc[:,'price_aprox_usd']
    
    tree = DecisionTreeRegressor(criterion = 'mse', max_depth = 100, max_features = 0.8)
    
    tree.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
    set_pruebas.loc[condiciones,'price_usd'] = set_pruebas.loc[condiciones,'price_usd'] + \
            set_pruebas.loc[condiciones,columnas].apply(lambda x: tree.predict([x])[0],axis = 1)
    print(i,end = '-')

set_pruebas.loc[condiciones,'price_usd'] = set_pruebas.loc[condiciones,'price_usd'] / cant

0-1-2-3-4-5-6-7-8-9-10-11-12-13-14-

# Calculo para aquellos que solo tienen superficie cubierta

In [6]:
condiciones = (set_pruebas.surface_covered_in_m2.notnull()) & (set_pruebas.price_usd == 0)
cond_prop = (propiedades.surface_covered_in_m2.notnull()) &\
        (propiedades.surface_covered_in_m2 <= sup_max) & (propiedades.surface_covered_in_m2 >= sup_min)

In [7]:
cant = 15
columnas = ['surface_covered_in_m2','lat', 'lon','property_type_encoded']

for i in range(cant):
    datos = propiedades.loc[cond_prop,:].sample(frac = 0.8)
    set_entrenamiento_datos = datos.loc[:,columnas]
    set_entrenamiento_resultado = datos.loc[:,'price_aprox_usd']
    
    tree = DecisionTreeRegressor(criterion = 'mse', max_depth = 100, max_features = 0.8)
    
    tree.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
    set_pruebas.loc[condiciones,'price_usd'] = set_pruebas.loc[condiciones,'price_usd'] + \
            set_pruebas.loc[condiciones,columnas].apply(lambda x: tree.predict([x])[0],axis = 1)
    print(i,end = '-')

set_pruebas.loc[condiciones,'price_usd'] = set_pruebas.loc[condiciones,'price_usd'] / cant

0-1-2-3-4-5-6-7-8-9-10-11-12-13-14-

# Calculo para aquellos que solo tienen superficie total

In [8]:
condiciones = (set_pruebas.surface_total_in_m2.notnull()) & (set_pruebas.price_usd == 0)
cond_prop = (propiedades.surface_total_in_m2.notnull()) &\
            (propiedades.surface_total_in_m2 <= sup_max) & (propiedades.surface_total_in_m2 >= sup_min)

In [9]:
cant = 15
columnas = ['surface_total_in_m2','lat', 'lon','property_type_encoded','gimnasio','pileta']

for i in range(cant):
    datos = propiedades.loc[cond_prop,:].sample(frac = 0.8)
    set_entrenamiento_datos = datos.loc[:,columnas]
    set_entrenamiento_resultado = datos.loc[:,'price_aprox_usd']
    
    tree = DecisionTreeRegressor(criterion = 'mse', max_depth = 100, max_features = 0.8)
    
    tree.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
    set_pruebas.loc[condiciones,'price_usd'] = set_pruebas.loc[condiciones,'price_usd'] + \
            set_pruebas.loc[condiciones,columnas].apply(lambda x: tree.predict([x])[0],axis = 1)
    print(i,end = '-')

set_pruebas.loc[condiciones,'price_usd'] = set_pruebas.loc[condiciones,'price_usd'] / cant

0-1-2-3-4-5-6-7-8-9-10-11-12-13-14-

# Calculo para aquellos que no tienen superficie

In [10]:
condiciones =  (set_pruebas.price_usd == 0)

In [11]:
cant = 15
columnas = ['lat', 'lon','property_type_encoded','pileta']

for i in range(cant):
    datos = propiedades.sample(frac = 0.8)
    set_entrenamiento_datos = datos.loc[:,columnas]
    set_entrenamiento_resultado = datos.loc[:,'price_aprox_usd']
    
    tree = DecisionTreeRegressor(criterion = 'mse', max_depth = 100, max_features = 0.8)
    
    tree.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
    set_pruebas.loc[condiciones,'price_usd'] = set_pruebas.loc[condiciones,'price_usd'] + \
            set_pruebas.loc[condiciones,columnas].apply(lambda x: tree.predict([x])[0],axis = 1)
    print(i,end = '-')

set_pruebas.loc[condiciones,'price_usd'] = set_pruebas.loc[condiciones,'price_usd'] / cant

0-1-2-3-4-5-6-7-8-9-10-11-12-13-14-

### Resultado final

In [13]:
resultado = set_pruebas.loc[:,['id','price_usd']]

resultado.to_csv('resultados/Decision_tree_superficies.csv', index = False)
resultado.describe()

,id,price_usd
count,1.416600e+04,1.416600e+04
mean,2.285382e+06,2.070423e+05
std,4.290968e+04,1.969797e+05
min,3.632000e+03,3.000000e+04
25%,2.271809e+06,9.966667e+04
50%,2.290284e+06,1.447974e+05
75%,2.309395e+06,2.367954e+05
max,2.332338e+06,2.130000e+06
